In [1]:
import time
from typing import Any, Dict, List, Tuple, Union

import argparse
import joblib
import tensorflow as tf
from termcolor import cprint
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils.baseline_utils as baseline_utils
from lstm_train_test import *

In [3]:
import pickle
import torch
import sys

sys.path.append('.')
sys.path.append('..')
import os
import numpy as np
from train_utils import *
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.patches import Ellipse
from datasets.argoverse_lane_loader import read_pkl_data
from argoverse.map_representation.map_api import ArgoverseMap
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datasets import helper

from tqdm import tqdm
from collections import defaultdict
from typing import Dict, Optional
from datasets.argoverse_lane_loader import read_pkl_data
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
from argoverse.map_representation.map_api import ArgoverseMap

# LSTM

In [9]:
device = 'cuda:0'

In [10]:
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=5)
encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)
encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

model_utils = ModelUtils()

model_path = '../checkpoints/lstm_argo_rot2/LSTM_rollout30.pth.tar'
epoch, rollout_len, _ = model_utils.load_checkpoint(
            model_path, encoder, decoder, encoder_optimizer,
            decoder_optimizer)

=> loading checkpoint '../checkpoints/lstm_argo_rot2/LSTM_rollout30.pth.tar'
=> loaded checkpoint ../checkpoints/lstm_argo_rot2/LSTM_rollout30.pth.tar (epoch: 115, loss: 3.148224115371704)


In [11]:
encoder.eval()
decoder.eval()

DataParallel(
  (module): DecoderRNN(
    (linear1): Linear(in_features=5, out_features=8, bias=True)
    (lstm1): LSTMCell(8, 32)
    (linear2): Linear(in_features=32, out_features=5, bias=True)
  )
)

In [ ]:
val_dataset = read_pkl_data('/data/sophiasun/argoverse/val', batch_size=1, shuffle=False, repeat=False)
for i, sample in enumerate(val_dataset):
   
    result = predict_lstm(sample)
    plot_lstm(sample, result)